In [1]:
import modules.models.model_torch_CNV_w1a1_Balanced_Resnet_bram as cnv_model
import modules.models.model_CNV_w1a1_Balanced_Resnet_bram as brevitas_cnv_model
from brevitas.export import export_qonnx

from torchinfo import summary
import torch

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


In [2]:
model = cnv_model.Torch_CNV_w1a1_Balanced_Resnet_bram()

In [3]:
model.to('cpu')

Torch_CNV_w1a1_Balanced_Resnet_bram(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): Conv2d(8, 14, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): ConvBlock(
      (conv_1): Sequential(
        (0): Conv2d(14, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(16, 28, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias

# Torchsummary

In [4]:
print(summary(model, input_size=(1, 3, 224, 224)))

Layer (type:depth-idx)                   Output Shape              Param #
Torch_CNV_w1a1_Balanced_Resnet_bram      [1, 2]                    --
├─Sequential: 1-1                        [1, 240, 1, 1]            --
│    └─Sequential: 2-1                   [1, 8, 112, 112]          --
│    │    └─Conv2d: 3-1                  [1, 8, 224, 224]          216
│    │    └─BatchNorm2d: 3-2             [1, 8, 224, 224]          16
│    │    └─ReLU: 3-3                    [1, 8, 224, 224]          --
│    │    └─MaxPool2d: 3-4               [1, 8, 112, 112]          --
│    └─Sequential: 2-2                   [1, 14, 56, 56]           --
│    │    └─Conv2d: 3-5                  [1, 14, 112, 112]         1,008
│    │    └─BatchNorm2d: 3-6             [1, 14, 112, 112]         28
│    │    └─ReLU: 3-7                    [1, 14, 112, 112]         --
│    │    └─MaxPool2d: 3-8               [1, 14, 56, 56]           --
│    └─ConvBlock: 2-3                    [1, 28, 28, 28]           --
│    │    └

# Export to ONNX

In [5]:
# model.to('cpu');

In [6]:
# torch_input = torch.randn(1, 3, 224, 224).to('cpu')
# torch.onnx.export(model, 
#                   torch_input, 
#                   "800K_w1a1_resnet.onnx",
#                   do_constant_folding=False,)

# Brevitas Model

In [7]:
brevitas_model = brevitas_cnv_model.CNV_w1a1_Balanced_Resnet_bram()

In [8]:
print(summary(brevitas_model, input_size=(1, 3, 224, 224)))

Layer (type:depth-idx)                                                           Output Shape              Param #
CNV_w1a1_Balanced_Resnet_bram                                                    [1, 2]                    --
├─Sequential: 1-1                                                                [1, 240, 1, 1]            749,740
│    └─QuantIdentity: 2-1                                                        [1, 3, 224, 224]          --
│    │    └─ActQuantProxyFromInjector: 3-1                                       [1, 3, 224, 224]          --
│    │    └─ActQuantProxyFromInjector: 3-2                                       [1, 3, 224, 224]          1
├─Sequential: 1-4                                                                --                        (recursive)
│    └─QuantLinear: 2-56                                                         --                        (recursive)
│    │    └─WeightQuantProxyFromInjector: 3-87                                   --          

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/torch/_tensor.py:1362: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400410390/work/c10/core/TensorImpl.h:1900.)
  return super().rename(names)


In [9]:
# brevitas_model.eval()
# brevitas_model.cpu()
# torch_input = torch.randn(1, 3, 224, 224).to('cpu')
# export_qonnx(brevitas_model, torch_input, 'Brevitas_Resnet_bram__QONNX.onnx');

In [13]:
for param in brevitas_model.features.parameters():
    print("New Param")
    print(param)

New Param
Parameter containing:
tensor(1., device='cuda:0', requires_grad=True)
New Param
Parameter containing:
tensor([[[[ 0.3099,  0.7068, -0.3528],
          [-0.8426,  0.9286,  0.7833],
          [ 0.8789,  0.6000,  0.4727]],

         [[ 0.6166,  0.6188, -0.4302],
          [ 0.1126, -0.0620, -0.4233],
          [ 0.7018, -0.8043,  0.8695]],

         [[-0.8801, -0.0209,  0.9022],
          [-0.2561, -0.9048,  0.4739],
          [-0.4154,  0.5836, -0.5284]]],


        [[[-0.8161,  0.9931, -0.5083],
          [ 0.9456, -0.9704, -0.3648],
          [ 0.9469,  0.1471,  0.0604]],

         [[-0.6436, -0.4255, -0.1005],
          [ 0.6263, -0.0698, -0.9494],
          [-0.2577, -0.7741,  0.0165]],

         [[ 0.5310,  0.5638,  0.0043],
          [-0.5476, -0.1305, -0.9708],
          [-0.1975, -0.4678,  0.5477]]],


        [[[ 0.1734,  0.6474,  0.9868],
          [ 0.0981, -0.5711,  0.4330],
          [ 0.9823, -0.6289,  0.3878]],

         [[-0.6334,  0.1870,  0.9951],
          [ 